# Generate Sentiment for subsetting data

This notebook generates sentiment scores for the raw data. These scores are used to subset the data so there is less data to manually label.


In [ ]:
GEO = "texas"

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

c:\Users\grego\Documents\GitHub\DS440CapstoneIndubitably\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import pandas as pd

reviews = pd.read_csv(f"../../data/raw/{GEO}_reviews.csv")
reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2265,963,2009-03-17,7538,Niall,I stayed here during SXSW and had a really ple...
1,2265,1057,2009-03-22,10029,Michael,"Great place, close enough to everything downto..."
2,2265,200418,2011-03-16,61677,Gustaf,We had a great time in Austin staying at Paddy...
3,2265,1001630,2012-03-15,1523753,Noah,We had a great stay at Zen East for South By S...
4,2265,1016390,2012-03-19,1547660,Melissa,I arrived late in the evening so did not meet ...


In [6]:
def get_sentiment(row):
    comment = row['comments']
    print(row.name)
    try:
        if len(comment) > 512:
            segments = [comment[i:i + 512] for i in range(0,len(comment),512)]
        else:
            segments = [comment]
        preds = []
        for segment in segments:
            tokens = tokenizer.encode(segment, return_tensors='pt')
            result = model(tokens)
            pred = int(torch.argmax(result.logits))+1
            preds.append(pred)
    
        return sum(preds)/len(preds)
    except:
        print("error")


sentiment_score = reviews.head(10).apply(get_sentiment,axis = 1)
reviews['sentiment'] = sentiment_score
reviews.to_csv("../../data/sentiment/texas_w_sentiment.csv",index = False)
reviews.describe()

0
1
2
3
4
5
6
7
8
9


,listing_id,id,reviewer_id,sentiment
count,3.320980e+05,3.320980e+05,3.320980e+05,10.000000
mean,1.275956e+07,2.504253e+08,7.330480e+07,4.600000
std,9.806012e+06,1.434814e+08,6.874664e+07,0.875595
min,2.265000e+03,8.650000e+02,3.000000e+00,2.500000
25%,3.403516e+06,1.305739e+08,1.707155e+07,5.000000
50%,1.226087e+07,2.517846e+08,5.044884e+07,5.000000
75%,2.075599e+07,3.658173e+08,1.155695e+08,5.000000
max,3.651485e+07,4.859486e+08,2.748575e+08,5.000000
